## Magnitude error from Howell 2006:
## $\sigma_{magnitudes}=\frac{1.0857 \sqrt{N_* + p}}{N_*} $
## $p = n_{pix} (1+ \frac{n_{pix}}{n_B})(N_S + N_D + N_R^2 +G^2\sigma_f^2)$
#### $n_{pix} =$ number of pixels in source region
#### $n_B =$ number of background pixels used to estimate the meal level of the background
#### $N_S =$ total number of photons per pixel from the background/sky
#### $N_D =$ total number of dark current electrons per pixel
#### $N_R^2 =$ total number of electrons per pixel resulting from the read noise
#### $G =$ gain of the CCD
#### $\sigma_f^2 =$ 1 sigma error introduced within the A/D converter ~ 0.289

In [3]:
import numpy as np

def snr(N_star, n_pix, n_B, N_S, N_D, N_R, G, sigma_f=0.289):
    p = n_pix * (1 + n_pix / n_B) * (N_S + N_D + N_R**2 + G**2 * sigma_f**2)
    snr_value = N_star / np.sqrt(N_star + p)
    return snr_value

def snr_est(N_star):
    return np.sqrt(N_star)

def mag_error(snr):
    return 1.0857 / snr

def photon_counts_per_sec(zpmag, target_mag):
    return 2.5**(zpmag - target_mag)

## RLMT Estimates

In [4]:
from glob import glob
from astropy.io import fits 
i_images = glob(r"E:\MACRO\MCV\Images\*\*\*_i_*.fts")
i_images_2 = glob(r"E:\MACRO\MCV\Images\*\*_i_*.fts")
print(len(i_images))
print(len(i_images_2))

zpmag = 0
num_included = 0
for img in i_images:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
for img in i_images_2:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1

if num_included > 0:
    print("Average ZPMAG:", zpmag / num_included)
else:
    print("No ZPMAG values found in the headers.")

43
3
No ZPMAG values found in the headers.


In [5]:
g_images = glob(r"E:\MACRO\MCV\Images\*\*\*_g_*.fts")
g_images_2 = glob(r"E:\MACRO\MCV\Images\*\*_g_*.fts")
print(len(g_images))
print(len(g_images_2))

zpmag = 0
num_included = 0
for img in g_images:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
for img in g_images_2:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
print("Average ZPMAG:", zpmag / num_included)

6
0
Average ZPMAG: 21.84211773392965


In [6]:
r_images = glob(r"E:\MACRO\MCV\Images\*\*\*_r_*.fts")
r_images_2 = glob(r"E:\MACRO\MCV\Images\*\*_r_*.fts")
print(len(r_images))
print(len(r_images_2))

zpmag = 0
num_included = 0
for img in r_images:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
for img in r_images_2:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
print("Average ZPMAG:", zpmag / num_included)

57
0
Average ZPMAG: 20.839567034245288


## Photon/electron counts for a given mag difference

In [7]:
photon_counts_per_mag_diff_per_sec = [2.5**num for num in range(1, 21)]
photon_counts_per_mag_diff_per_sec = np.array(photon_counts_per_mag_diff_per_sec)
print(photon_counts_per_mag_diff_per_sec)

[2.50000000e+00 6.25000000e+00 1.56250000e+01 3.90625000e+01
 9.76562500e+01 2.44140625e+02 6.10351562e+02 1.52587891e+03
 3.81469727e+03 9.53674316e+03 2.38418579e+04 5.96046448e+04
 1.49011612e+05 3.72529030e+05 9.31322575e+05 2.32830644e+06
 5.82076609e+06 1.45519152e+07 3.63797881e+07 9.09494702e+07]


## Associated mag error estimates

In [8]:
snr_1s = snr_est(photon_counts_per_mag_diff_per_sec * 1)
snr_10s = snr_est(photon_counts_per_mag_diff_per_sec * 10)
snr_30s = snr_est(photon_counts_per_mag_diff_per_sec * 30)
snr_60s = snr_est(photon_counts_per_mag_diff_per_sec * 60)
snr_120s = snr_est(photon_counts_per_mag_diff_per_sec * 120)
snr_300s = snr_est(photon_counts_per_mag_diff_per_sec * 300)

est_mag_errors_1s = mag_error(snr_1s)
est_mag_errors_10s = mag_error(snr_10s) 
est_mag_errors_30s = mag_error(snr_30s)
est_mag_errors_60s = mag_error(snr_60s)
est_mag_errors_120s = mag_error(snr_120s)
est_mag_errors_300s = mag_error(snr_300s)

for i in range(len(snr_1s)):
    print(f"Mag difference: {i+1}, SNR for 1s: {snr_1s[i]:.2f}, Mag error for 1s: {est_mag_errors_1s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 10s: {snr_10s[i]:.2f}, Mag error for 10s: {est_mag_errors_10s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 30s: {snr_30s[i]:.2f}, Mag error for 30s: {est_mag_errors_30s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 60s: {snr_60s[i]:.2f}, Mag error for 60s: {est_mag_errors_60s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 120s: {snr_120s[i]:.2f}, Mag error for 120s: {est_mag_errors_120s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 300s: {snr_300s[i]:.2f}, Mag error for 300s: {est_mag_errors_300s[i]:.4f}")
    print("")

Mag difference: 1, SNR for 1s: 1.58, Mag error for 1s: 0.6867
Mag difference: 1, SNR for 10s: 5.00, Mag error for 10s: 0.2171
Mag difference: 1, SNR for 30s: 8.66, Mag error for 30s: 0.1254
Mag difference: 1, SNR for 60s: 12.25, Mag error for 60s: 0.0886
Mag difference: 1, SNR for 120s: 17.32, Mag error for 120s: 0.0627
Mag difference: 1, SNR for 300s: 27.39, Mag error for 300s: 0.0396

Mag difference: 2, SNR for 1s: 2.50, Mag error for 1s: 0.4343
Mag difference: 2, SNR for 10s: 7.91, Mag error for 10s: 0.1373
Mag difference: 2, SNR for 30s: 13.69, Mag error for 30s: 0.0793
Mag difference: 2, SNR for 60s: 19.36, Mag error for 60s: 0.0561
Mag difference: 2, SNR for 120s: 27.39, Mag error for 120s: 0.0396
Mag difference: 2, SNR for 300s: 43.30, Mag error for 300s: 0.0251

Mag difference: 3, SNR for 1s: 3.95, Mag error for 1s: 0.2747
Mag difference: 3, SNR for 10s: 12.50, Mag error for 10s: 0.0869
Mag difference: 3, SNR for 30s: 21.65, Mag error for 30s: 0.0501
Mag difference: 3, SNR for 

## $\Delta m = 21 - 2.5log_{10}(photons/s \pm transit\ depth \%)$

In [9]:
transit_depths = [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 25.0, 50.0]  # in percentage
for depth in transit_depths:
    print(f"Transit depth: {depth}%")
    for i, counts in enumerate(photon_counts_per_mag_diff_per_sec):
        m = 21 - 2.5 * np.log10(counts)
        delta_m_P = 21 - 2.5 * np.log10(counts * (1 + depth / 100)) - m
        delta_m_M = 21 - 2.5 * np.log10(counts * (1 - depth / 100)) - m

        print(f"Mag difference: {i+1}, Δm+ : {delta_m_P:.4f}, Δm- : {delta_m_M:.4f}, m: {m:.4f}")
    print("")

Transit depth: 0.01%
Mag difference: 1, Δm+ : -0.0001, Δm- : 0.0001, m: 20.0051
Mag difference: 2, Δm+ : -0.0001, Δm- : 0.0001, m: 19.0103
Mag difference: 3, Δm+ : -0.0001, Δm- : 0.0001, m: 18.0154
Mag difference: 4, Δm+ : -0.0001, Δm- : 0.0001, m: 17.0206
Mag difference: 5, Δm+ : -0.0001, Δm- : 0.0001, m: 16.0257
Mag difference: 6, Δm+ : -0.0001, Δm- : 0.0001, m: 15.0309
Mag difference: 7, Δm+ : -0.0001, Δm- : 0.0001, m: 14.0360
Mag difference: 8, Δm+ : -0.0001, Δm- : 0.0001, m: 13.0412
Mag difference: 9, Δm+ : -0.0001, Δm- : 0.0001, m: 12.0463
Mag difference: 10, Δm+ : -0.0001, Δm- : 0.0001, m: 11.0515
Mag difference: 11, Δm+ : -0.0001, Δm- : 0.0001, m: 10.0566
Mag difference: 12, Δm+ : -0.0001, Δm- : 0.0001, m: 9.0618
Mag difference: 13, Δm+ : -0.0001, Δm- : 0.0001, m: 8.0669
Mag difference: 14, Δm+ : -0.0001, Δm- : 0.0001, m: 7.0721
Mag difference: 15, Δm+ : -0.0001, Δm- : 0.0001, m: 6.0772
Mag difference: 16, Δm+ : -0.0001, Δm- : 0.0001, m: 5.0824
Mag difference: 17, Δm+ : -0.0001

## Estimating 10% flux on center pixel

In [10]:
full_well = 65535.0
counts_per_sec_10_percent = photon_counts_per_mag_diff_per_sec*0.1
saturation_times = full_well / counts_per_sec_10_percent
for i, time in enumerate(saturation_times):
    print(f"Mag difference: {i+1}, Saturation time (s): {time:.2f}")

Mag difference: 1, Saturation time (s): 262140.00
Mag difference: 2, Saturation time (s): 104856.00
Mag difference: 3, Saturation time (s): 41942.40
Mag difference: 4, Saturation time (s): 16776.96
Mag difference: 5, Saturation time (s): 6710.78
Mag difference: 6, Saturation time (s): 2684.31
Mag difference: 7, Saturation time (s): 1073.73
Mag difference: 8, Saturation time (s): 429.49
Mag difference: 9, Saturation time (s): 171.80
Mag difference: 10, Saturation time (s): 68.72
Mag difference: 11, Saturation time (s): 27.49
Mag difference: 12, Saturation time (s): 10.99
Mag difference: 13, Saturation time (s): 4.40
Mag difference: 14, Saturation time (s): 1.76
Mag difference: 15, Saturation time (s): 0.70
Mag difference: 16, Saturation time (s): 0.28
Mag difference: 17, Saturation time (s): 0.11
Mag difference: 18, Saturation time (s): 0.05
Mag difference: 19, Saturation time (s): 0.02
Mag difference: 20, Saturation time (s): 0.01


## Estimating 25% flux on center pixel

In [11]:
saturation_times_25 = full_well / (photon_counts_per_mag_diff_per_sec*0.25)
for i, time in enumerate(saturation_times_25):
    print(f"Mag difference: {i+1}, Saturation time (s) at 25% flux: {time:.2f}")

Mag difference: 1, Saturation time (s) at 25% flux: 104856.00
Mag difference: 2, Saturation time (s) at 25% flux: 41942.40
Mag difference: 3, Saturation time (s) at 25% flux: 16776.96
Mag difference: 4, Saturation time (s) at 25% flux: 6710.78
Mag difference: 5, Saturation time (s) at 25% flux: 2684.31
Mag difference: 6, Saturation time (s) at 25% flux: 1073.73
Mag difference: 7, Saturation time (s) at 25% flux: 429.49
Mag difference: 8, Saturation time (s) at 25% flux: 171.80
Mag difference: 9, Saturation time (s) at 25% flux: 68.72
Mag difference: 10, Saturation time (s) at 25% flux: 27.49
Mag difference: 11, Saturation time (s) at 25% flux: 10.99
Mag difference: 12, Saturation time (s) at 25% flux: 4.40
Mag difference: 13, Saturation time (s) at 25% flux: 1.76
Mag difference: 14, Saturation time (s) at 25% flux: 0.70
Mag difference: 15, Saturation time (s) at 25% flux: 0.28
Mag difference: 16, Saturation time (s) at 25% flux: 0.11
Mag difference: 17, Saturation time (s) at 25% flux: